In [1]:
import logging
from sacred import Experiment
import seml
import os
from transVAE.model.transVAE import transVAE
from transVAE.module._utils import check_adatas_var_index, compare_de, compare_logfold, compute_r2_score
import scanpy as sc
import scvi
import numpy as np
import gc
import pandas as pd


/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


In [63]:
# Fixed values
max_epochs = 10
dataset_to_predict = "wang_to_predict_cleanCT.h5ad"
dataset_ground_truth2 = "mSTZ_ground_truth_cleanCT.h5ad"
dataset_ground_truth1 = "dbdb_ground_truth_cleanCT.h5ad"
cov_embed_dims = 10
kl_weight = 0.005
save_folder_name = "test_final"
save_models = True
learning_rate = 0.0001
n_latent = 512
n_hidden = 1000

# Grid values (manually selected)
addl_dataset_train = "extra_human_chem.h5ad"
n_layers = 6  # Chosen from range 6 to 10 with step 2
dropout_rate = 0.3  # Chosen from options 0.3 and 0.4
weight_decay = 0.001  # Chosen from options 0.0001 and 0.001
seed = 42  # Chosen from options 42, 43, and 44

if addl_dataset_train == "train_adata_baseline_3000hvg.h5ad":
    adata_train = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/top3000_just_baseline/train_adata_baseline_3000hvg.h5ad")
    logging.info(f'Received the following configuration for datasets train: {addl_dataset_train}, shape: {adata_train.shape}')
else:
    adata_baseline = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/top3000_just_baseline/train_adata_baseline_3000hvg.h5ad")
    adata_extra = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/top3000_just_baseline/{addl_dataset_train}")
    adata_train = sc.concat([adata_baseline, adata_extra], join = "inner", axis = 0)
    logging.info(f'Received the following configuration for datasets train: {addl_dataset_train}, shape: {adata_train.shape}, baseline_shape: {adata_baseline.shape}')
    del adata_baseline
    del adata_extra
    gc.collect()

adata_train = transVAE.setup_anndata(adata_train, categorical_covariate_embed_keys=["dataset"], categorical_covariate_keys=["organism"], copy = True)
adata_to_predict = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/data_to_predict/{dataset_to_predict}")
dbdb_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/{dataset_ground_truth1}")
mSTZ_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/{dataset_ground_truth2}")

scvi.settings.seed = seed

logging.info(f'n_hidden: {n_hidden}, n_latent: {n_latent}, learning_rate: {learning_rate}, cov_embed_dims: {cov_embed_dims}, weight_decay {weight_decay}')
logging.info(f'max_epochs: {max_epochs}, n_layers: {n_layers}, dropout_rate: {dropout_rate}, kl_weight: {kl_weight}')

check_adatas_var_index(adata_train, adata_to_predict, dbdb_ground_truth, mSTZ_ground_truth)

adata_to_predict_h = adata_to_predict[adata_to_predict.obs.disease == "normal"]
adata_to_predict_OOD = adata_to_predict[adata_to_predict.obs.disease == "T2D"]
del adata_to_predict
gc.collect()

dbdb_ground_truth_h = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "normal"]
dbdb_ground_truth_OOD = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "T2D"]
del dbdb_ground_truth
gc.collect()

mSTZ_ground_truth_h = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "normal"]
mSTZ_ground_truth_OOD = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "T2D"]
del mSTZ_ground_truth
gc.collect()

validation_adatas_dict = {"adata_to_predict":adata_to_predict_OOD,
                          "ground_truths": {"dbdb": dbdb_ground_truth_OOD,
                                            "mSTZ": mSTZ_ground_truth_OOD},
                          "translate_dicts": {"dbdb":{"dataset":"db/db", "organism": "Mus musculus"},
                                              "mSTZ":{"dataset":"mSTZ", "organism": "Mus musculus"}}}


# make the model
model = transVAE(adata_train, 
                 n_hidden=n_hidden, 
                 n_latent=n_latent, 
                 n_layers=n_layers, 
                 dropout_rate=dropout_rate, 
                 cov_embed_dims = cov_embed_dims, 
                 kl_weight = kl_weight,
                 validation_adatas_dict = validation_adatas_dict)


# train the model
model.train(batch_size=4096, max_epochs = max_epochs, train_size = 0.99, enable_progress_bar = True,
            early_stopping = True, early_stopping_monitor = 'mean_r2_validation_eval', early_stopping_mode = "max", 
            early_stopping_min_delta = 0.01, early_stopping_patience = 70,
            plan_kwargs = {"lr":learning_rate,
                           "weight_decay":weight_decay,
                           "reduce_lr_on_plateau":True,
                           "lr_factor":0.5,
                           "lr_patience":50,
                           "lr_scheduler_metric":"reconstruction_loss_validation"})

results = model.history

# dbdb
preds_dbdb_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})
preds_dbdb_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})

# mSTZ
preds_mSTZ_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})
preds_mSTZ_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})

# dbdb
results["r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h, dbdb_ground_truth_h)
results["r2_dbdb_OOD"] = model.history["dbdb_r2_validation_eval"].iloc[-1].item()
# mSTZ
results["r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h, mSTZ_ground_truth_h)
results["r2_mSTZ_OOD"] = model.history["mSTZ_r2_validation_eval"].iloc[-1].item()

results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD.X.toarray(), Y=preds_dbdb_OOD.X, C=dbdb_ground_truth_h.X.toarray(), shared_top = 1000, method = "t-test")
results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD.X.toarray(), Y=preds_mSTZ_OOD.X, C=mSTZ_ground_truth_h.X.toarray(), shared_top = 1000, method = "t-test")

for name, value in results_de_dbdb.items():
    results[name + "_dbdb"] = value
for name, value in results_de_mSTZ.items():
    results[name + "_mSTZ"] = value

for ct in dbdb_ground_truth_OOD.obs.cell_type.unique():
    
    # healthy splits
    preds_dbdb_h_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
    dbdb_ground_truth_h_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
    preds_mSTZ_h_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
    mSTZ_ground_truth_h_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]
    
    # OOD splits
    preds_dbdb_OOD_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
    dbdb_ground_truth_OOD_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
    preds_mSTZ_OOD_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
    mSTZ_ground_truth_OOD_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]
    
    ct = ct.replace(" ", "_")
    
    results[ct + "_" + "r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h_tmp, dbdb_ground_truth_h_tmp)
    results[ct + "_" + "r2_dbdb_OOD"] = compute_r2_score(preds_mSTZ_OOD_tmp, mSTZ_ground_truth_OOD_tmp)
    # mSTZ
    results[ct + "_" + "r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h_tmp, mSTZ_ground_truth_h_tmp)
    results[ct + "_" + "r2_mSTZ_OOD"] = compute_r2_score(preds_mSTZ_OOD_tmp, mSTZ_ground_truth_OOD_tmp)

    results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD_tmp.X.toarray(), Y=preds_dbdb_OOD_tmp.X, C=dbdb_ground_truth_h_tmp.X.toarray(), shared_top = 1000, method = "t-test")
    results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD_tmp.X.toarray(), Y=preds_mSTZ_OOD_tmp.X, C=mSTZ_ground_truth_h_tmp.X.toarray(), shared_top = 1000, method = "t-test")

    for name, value in results_de_dbdb.items():
        results[ct + "_" + name + "_dbdb" ] = value
    for name, value in results_de_mSTZ.items():
        results[ct + "_" + name + "_mSTZ_"] = value

if save_models:
    save_dir = f"/d/hpc/projects/FRI/DL/mo6643/MSC/cross_species_prediction_save/{save_folder_name}"
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_dir = os.path.join(save_dir, f"transVAE_train_{addl_dataset_train}_hid{n_hidden}_lat{n_latent}_lr{learning_rate}_cov{cov_embed_dims}_ep{max_epochs}_ly{n_layers}_dr{dropout_rate}_kl{kl_weight}_wd{weight_decay}_s{seed}")
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    #np.save(arr=preds_dbdb_h.X, file=os.path.join(save_dir,"preds_dbdb_h.npy"))
    np.save(arr=preds_dbdb_OOD.X, file=os.path.join(save_dir,"preds_dbdb_OOD.npy"))
    #np.save(arr=preds_mSTZ_h.X, file=os.path.join(save_dir,"preds_mSTZ_h.npy"))
    np.save(arr=preds_mSTZ_OOD.X, file=os.path.join(save_dir,"preds_mSTZ_OOD.npy"))

In [57]:
genes_intersection = pd.read_table("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/make_big_dataset/genes_shared_across_datasets.txt")
genes_intersection = genes_intersection.genes_shared_across_datasets.tolist()

In [ ]:
if addl_dataset_train == "train_adata_baseline_3000hvg.h5ad":
    adata_train = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/train_adata_baseline.h5ad")
    logging.info(f'Received the following configuration for datasets train: {addl_dataset_train}, shape: {adata_train.shape}')
else:
    adata_baseline = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/train_adata_baseline.h5ad")
    adata_extra = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/{addl_dataset_train}")
    adata_train = sc.concat([adata_baseline, adata_extra], join = "inner", axis = 0)
    logging.info(f'Received the following configuration for datasets train: {addl_dataset_train}, shape: {adata_train.shape}, baseline_shape: {adata_baseline.shape}')
    del adata_baseline
    del adata_extra
    gc.collect()

genes_intersection = pd.read_table("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/make_big_dataset/genes_shared_across_datasets.txt")
genes_intersection = genes_intersection.genes_shared_across_datasets.tolist()

## making random splits
adata_train = adata_train[:,adata_train.var.index.isin(genes_intersection)]
adata_train = transVAE.setup_anndata(adata_train, categorical_covariate_embed_keys=["dataset"], categorical_covariate_keys=["organism"], copy = True)

# reading val data and data to translate
adata_to_predict = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/data_to_predict/{dataset_to_predict}")
dbdb_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth1}")
mSTZ_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth2}")

adata_to_predict = adata_to_predict[:,adata_to_predict.var.index.isin(genes_intersection)]
adata_to_predict = adata_to_predict[:,adata_to_predict.var.sort_index().index]
dbdb_ground_truth = dbdb_ground_truth[:,dbdb_ground_truth.var.index.isin(genes_intersection)]
mSTZ_ground_truth = mSTZ_ground_truth[:,mSTZ_ground_truth.var.index.isin(genes_intersection)]

scvi.settings.seed = seed

logging.info(f'n_hidden: {n_hidden}, n_latent: {n_latent}, learning_rate: {learning_rate}, cov_embed_dims: {cov_embed_dims}, weight_decay {weight_decay}')
logging.info(f'max_epochs: {max_epochs}, n_layers: {n_layers}, dropout_rate: {dropout_rate}, kl_weight: {kl_weight}')

check_adatas_var_index(adata_train, adata_to_predict, dbdb_ground_truth, mSTZ_ground_truth)

# check this data (full_datasets)

adata_to_predict_h = adata_to_predict[adata_to_predict.obs.disease == "healthy"]
adata_to_predict_OOD = adata_to_predict[adata_to_predict.obs.disease == "T2D"]
del adata_to_predict
gc.collect()

dbdb_ground_truth_h = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "healthy"]
dbdb_ground_truth_OOD = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "T2D"]
del dbdb_ground_truth
gc.collect()

mSTZ_ground_truth_h = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "healthy"]
mSTZ_ground_truth_OOD = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "T2D"]
del mSTZ_ground_truth
gc.collect()

validation_adatas_dict = {"adata_to_predict":adata_to_predict_OOD,
                          "ground_truths": {"dbdb": dbdb_ground_truth_OOD,
                                            "mSTZ": mSTZ_ground_truth_OOD},
                          "translate_dicts": {"dbdb":{"dataset":"db/db", "organism": "Mus musculus"},
                                              "mSTZ":{"dataset":"mSTZ", "organism": "Mus musculus"}}}


# make the model
model = transVAE(adata_train, 
                 n_hidden=n_hidden, 
                 n_latent=n_latent, 
                 n_layers=n_layers, 
                 dropout_rate=dropout_rate, 
                 cov_embed_dims = cov_embed_dims, 
                 kl_weight = kl_weight,
                 validation_adatas_dict = validation_adatas_dict)

# train the model
model.train(batch_size=4096, max_epochs = max_epochs, train_size = 0.99, enable_progress_bar = True,
            early_stopping = True, early_stopping_monitor = 'mean_r2_validation_eval', early_stopping_mode = "max", 
            early_stopping_min_delta = 0.01, early_stopping_patience = 70,
            plan_kwargs = {"lr":learning_rate,
                           "weight_decay":weight_decay,
                           "reduce_lr_on_plateau":True,
                           "lr_factor":0.5,
                           "lr_patience":50,
                           "lr_scheduler_metric":"reconstruction_loss_validation"})

results = model.history

# dbdb
preds_dbdb_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})
preds_dbdb_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})

# mSTZ
preds_mSTZ_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})
preds_mSTZ_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})

# dbdb
results["r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h, dbdb_ground_truth_h)
results["r2_dbdb_OOD"] = model.history["dbdb_r2_validation_eval"].iloc[-1].item()
# mSTZ
results["r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h, mSTZ_ground_truth_h)
results["r2_mSTZ_OOD"] = model.history["mSTZ_r2_validation_eval"].iloc[-1].item()

results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD.X.toarray(), Y=preds_dbdb_OOD.X, C=dbdb_ground_truth_h.X.toarray(), shared_top = 1000, method = "t-test")
results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD.X.toarray(), Y=preds_mSTZ_OOD.X, C=mSTZ_ground_truth_h.X.toarray(), shared_top = 1000, method = "t-test")

for name, value in results_de_dbdb.items():
    results[name + "_dbdb"] = value
for name, value in results_de_mSTZ.items():
    results[name + "_mSTZ"] = value

for ct in dbdb_ground_truth_OOD.obs.cell_type.unique():
    
    # healthy splits
    preds_dbdb_h_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
    dbdb_ground_truth_h_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
    preds_mSTZ_h_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
    mSTZ_ground_truth_h_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]
    
    # OOD splits
    preds_dbdb_OOD_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
    dbdb_ground_truth_OOD_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
    preds_mSTZ_OOD_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
    mSTZ_ground_truth_OOD_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]
    
    ct = ct.replace(" ", "_")
    
    results[ct + "_" + "r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h_tmp, dbdb_ground_truth_h_tmp)
    results[ct + "_" + "r2_dbdb_OOD"] = compute_r2_score(preds_mSTZ_OOD_tmp, mSTZ_ground_truth_OOD_tmp)
    # mSTZ
    results[ct + "_" + "r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h_tmp, mSTZ_ground_truth_h_tmp)
    results[ct + "_" + "r2_mSTZ_OOD"] = compute_r2_score(preds_mSTZ_OOD_tmp, mSTZ_ground_truth_OOD_tmp)

    results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD_tmp.X.toarray(), Y=preds_dbdb_OOD_tmp.X, C=dbdb_ground_truth_h_tmp.X.toarray(), shared_top = 1000, method = "t-test")
    results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD_tmp.X.toarray(), Y=preds_mSTZ_OOD_tmp.X, C=mSTZ_ground_truth_h_tmp.X.toarray(), shared_top = 1000, method = "t-test")

    for name, value in results_de_dbdb.items():
        results[ct + "_" + name + "_dbdb" ] = value
    for name, value in results_de_mSTZ.items():
        results[ct + "_" + name + "_mSTZ_"] = value

if save_models:
    save_dir = f"/d/hpc/projects/FRI/DL/mo6643/MSC/cross_species_prediction_save/{save_folder_name}"
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_dir = os.path.join(save_dir, f"transVAE_train_{addl_dataset_train}_hid{n_hidden}_lat{n_latent}_lr{learning_rate}_cov{cov_embed_dims}_ep{max_epochs}_ly{n_layers}_dr{dropout_rate}_kl{kl_weight}_wd{weight_decay}_s{seed}")
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    #np.save(arr=preds_dbdb_h.X, file=os.path.join(save_dir,"preds_dbdb_h.npy"))
    np.save(arr=preds_dbdb_OOD.X, file=os.path.join(save_dir,"preds_dbdb_OOD.npy"))
    #np.save(arr=preds_mSTZ_h.X, file=os.path.join(save_dir,"preds_mSTZ_h.npy"))
    np.save(arr=preds_mSTZ_OOD.X, file=os.path.join(save_dir,"preds_mSTZ_OOD.npy"))

return results

In [78]:
if addl_dataset_train == "train_adata_baseline_3000hvg.h5ad":
    adata_train = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/train_adata_baseline.h5ad")
    logging.info(f'Received the following configuration for datasets train: {addl_dataset_train}, shape: {adata_train.shape}')
else:
    adata_baseline = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/train_adata_baseline.h5ad")
    adata_extra = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/{addl_dataset_train}")
    adata_train = sc.concat([adata_baseline, adata_extra], join = "inner", axis = 0)
    logging.info(f'Received the following configuration for datasets train: {addl_dataset_train}, shape: {adata_train.shape}, baseline_shape: {adata_baseline.shape}')
    del adata_baseline
    del adata_extra
    gc.collect()

genes_intersection = pd.read_csv("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/top3000_hv_genes_gr_truth_and_baseline_across_datasets.csv")
genes_intersection = genes_intersection.EID.tolist()

## making random splits
adata_train = adata_train[:,adata_train.var.index.isin(genes_intersection)]
adata_train = transVAE.setup_anndata(adata_train, categorical_covariate_embed_keys=["dataset"], categorical_covariate_keys=["organism"], copy = True)



# reading val data and data to translate
adata_to_predict = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/data_to_predict/{dataset_to_predict}")
dbdb_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth1}")
mSTZ_ground_truth = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/ground_truth/full_datasets/{dataset_ground_truth2}")

adata_to_predict = adata_to_predict[:,adata_to_predict.var.index.isin(genes_intersection)]
adata_to_predict = adata_to_predict[:,adata_to_predict.var.sort_index().index]
dbdb_ground_truth = dbdb_ground_truth[:,dbdb_ground_truth.var.index.isin(genes_intersection)]
mSTZ_ground_truth = mSTZ_ground_truth[:,mSTZ_ground_truth.var.index.isin(genes_intersection)]

scvi.settings.seed = seed

logging.info(f'n_hidden: {n_hidden}, n_latent: {n_latent}, learning_rate: {learning_rate}, cov_embed_dims: {cov_embed_dims}, weight_decay {weight_decay}')
logging.info(f'max_epochs: {max_epochs}, n_layers: {n_layers}, dropout_rate: {dropout_rate}, kl_weight: {kl_weight}')

check_adatas_var_index(adata_train, adata_to_predict, dbdb_ground_truth, mSTZ_ground_truth)

# check this data (full_datasets)

adata_to_predict_h = adata_to_predict[adata_to_predict.obs.disease == "healthy"]
adata_to_predict_OOD = adata_to_predict[adata_to_predict.obs.disease == "T2D"]
del adata_to_predict
gc.collect()

dbdb_ground_truth_h = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "healthy"]
dbdb_ground_truth_OOD = dbdb_ground_truth[dbdb_ground_truth.obs.disease == "T2D"]
del dbdb_ground_truth
gc.collect()

mSTZ_ground_truth_h = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "healthy"]
mSTZ_ground_truth_OOD = mSTZ_ground_truth[mSTZ_ground_truth.obs.disease == "T2D"]
del mSTZ_ground_truth
gc.collect()

validation_adatas_dict = {"adata_to_predict":adata_to_predict_OOD,
                          "ground_truths": {"dbdb": dbdb_ground_truth_OOD,
                                            "mSTZ": mSTZ_ground_truth_OOD},
                          "translate_dicts": {"dbdb":{"dataset":"db/db", "organism": "Mus musculus"},
                                              "mSTZ":{"dataset":"mSTZ", "organism": "Mus musculus"}}}


# make the model
model = transVAE(adata_train, 
                 n_hidden=n_hidden, 
                 n_latent=n_latent, 
                 n_layers=n_layers, 
                 dropout_rate=dropout_rate, 
                 cov_embed_dims = cov_embed_dims, 
                 kl_weight = kl_weight,
                 validation_adatas_dict = validation_adatas_dict)

# train the model
model.train(batch_size=4096, max_epochs = max_epochs, train_size = 0.99, enable_progress_bar = True,
            early_stopping = True, early_stopping_monitor = 'mean_r2_validation_eval', early_stopping_mode = "max", 
            early_stopping_min_delta = 0.01, early_stopping_patience = 70,
            plan_kwargs = {"lr":learning_rate,
                           "weight_decay":weight_decay,
                           "reduce_lr_on_plateau":True,
                           "lr_factor":0.5,
                           "lr_patience":50,
                           "lr_scheduler_metric":"reconstruction_loss_validation"})

results = model.history

# dbdb
preds_dbdb_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})
preds_dbdb_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"db/db", "organism": "Mus musculus"})

# mSTZ
preds_mSTZ_h = model.translate(adata_to_predict_h, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})
preds_mSTZ_OOD = model.translate(adata_to_predict_OOD, translate_dict={"dataset":"mSTZ", "organism": "Mus musculus"})

# dbdb
results["r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h, dbdb_ground_truth_h)
results["r2_dbdb_OOD"] = model.history["dbdb_r2_validation_eval"].iloc[-1].item()
# mSTZ
results["r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h, mSTZ_ground_truth_h)
results["r2_mSTZ_OOD"] = model.history["mSTZ_r2_validation_eval"].iloc[-1].item()

results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD.X.toarray(), Y=preds_dbdb_OOD.X, C=dbdb_ground_truth_h.X.toarray(), shared_top = 1000, method = "t-test")
results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD.X.toarray(), Y=preds_mSTZ_OOD.X, C=mSTZ_ground_truth_h.X.toarray(), shared_top = 1000, method = "t-test")

for name, value in results_de_dbdb.items():
    results[name + "_dbdb"] = value
for name, value in results_de_mSTZ.items():
    results[name + "_mSTZ"] = value

for ct in dbdb_ground_truth_OOD.obs.cell_type.unique():

    # healthy splits
    preds_dbdb_h_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
    dbdb_ground_truth_h_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
    preds_mSTZ_h_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
    mSTZ_ground_truth_h_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]

    # OOD splits
    preds_dbdb_OOD_tmp = preds_dbdb_h[preds_dbdb_h.obs.cell_type == ct]
    dbdb_ground_truth_OOD_tmp = dbdb_ground_truth_h[dbdb_ground_truth_h.obs.cell_type == ct]
    preds_mSTZ_OOD_tmp = preds_mSTZ_h[preds_mSTZ_h.obs.cell_type == ct]
    mSTZ_ground_truth_OOD_tmp = mSTZ_ground_truth_h[mSTZ_ground_truth_h.obs.cell_type == ct]

    ct = ct.replace(" ", "_")

    results[ct + "_" + "r2_dbdb_healthy"] = compute_r2_score(preds_dbdb_h_tmp, dbdb_ground_truth_h_tmp)
    results[ct + "_" + "r2_dbdb_OOD"] = compute_r2_score(preds_mSTZ_OOD_tmp, mSTZ_ground_truth_OOD_tmp)
    # mSTZ
    results[ct + "_" + "r2_mSTZ_healthy"] = compute_r2_score(preds_mSTZ_h_tmp, mSTZ_ground_truth_h_tmp)
    results[ct + "_" + "r2_mSTZ_OOD"] = compute_r2_score(preds_mSTZ_OOD_tmp, mSTZ_ground_truth_OOD_tmp)

    results_de_dbdb = compare_de(X=dbdb_ground_truth_OOD_tmp.X.toarray(), Y=preds_dbdb_OOD_tmp.X, C=dbdb_ground_truth_h_tmp.X.toarray(), shared_top = 1000, method = "t-test")
    results_de_mSTZ = compare_de(X=mSTZ_ground_truth_OOD_tmp.X.toarray(), Y=preds_mSTZ_OOD_tmp.X, C=mSTZ_ground_truth_h_tmp.X.toarray(), shared_top = 1000, method = "t-test")

    for name, value in results_de_dbdb.items():
        results[ct + "_" + name + "_dbdb" ] = value
    for name, value in results_de_mSTZ.items():
        results[ct + "_" + name + "_mSTZ_"] = value

if save_models:
    try:
        save_dir = f"/d/hpc/projects/FRI/DL/mo6643/MSC/cross_species_prediction_save/{save_folder_name}"
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_dir = os.path.join(save_dir, f"transVAE_train_{addl_dataset_train}_hid{n_hidden}_lat{n_latent}_lr{learning_rate}_cov{cov_embed_dims}_ep{max_epochs}_ly{n_layers}_dr{dropout_rate}_kl{kl_weight}_wd{weight_decay}_s{seed}")
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        #np.save(arr=preds_dbdb_h.X, file=os.path.join(save_dir,"preds_dbdb_h.npy"))
        np.save(arr=preds_dbdb_OOD.X, file=os.path.join(save_dir,"preds_dbdb_OOD.npy"))
        #np.save(arr=preds_mSTZ_h.X, file=os.path.join(save_dir,"preds_mSTZ_h.npy"))
        np.save(arr=preds_mSTZ_OOD.X, file=os.path.join(save_dir,"preds_mSTZ_OOD.npy"))
    except:
        print("An error accured while saving model predictions")

In [83]:
adata_train.obs.dataset.unique()

array(['mSTZ', '4m', 'db/db', 'Elgmal', 'Wang', 'chem'], dtype=object)

In [84]:
adata_train.obs.disease.unique()

array(['normal', 'T2D'], dtype=object)

In [85]:
adata_train.obs.organism.unique()

array(['Mus musculus', 'Homo sapiens'], dtype=object)

In [97]:
adata_train.obs

,disease,dataset,organism,cell_type,n_counts
HPAP-051_AAACGCTTCGGCTTGG-1,T2D,Elgmal,Homo sapiens,Ductal,29567.0
HPAP-051_AAAGGATAGACTTCAC-1,T2D,Elgmal,Homo sapiens,Beta,14308.0
HPAP-051_AAAGGATCATAAGCGG-1,T2D,Elgmal,Homo sapiens,Alpha,3694.0
HPAP-051_AAAGGGCGTCCCACGA-1,T2D,Elgmal,Homo sapiens,Alpha,6267.0
HPAP-051_AAAGGTACATTGCCGG-1,T2D,Elgmal,Homo sapiens,Alpha,11056.0
...,...,...,...,...,...
C0024_TTTGTGTTCCCTCGCA-1,T2D,Wang,Homo sapiens,alpha,3479.0
C0024_TTTGTGTTCTTGCAGG-1,T2D,Wang,Homo sapiens,alpha,4410.0
C0024_TTTGTTGGTATTGTCC-1,T2D,Wang,Homo sapiens,alpha,6104.0
C0024_TTTGTTGGTCATTACC-1,T2D,Wang,Homo sapiens,beta,4908.0


In [99]:
adata_train = sc.read_h5ad(f"/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/full_datasets/train_adata_baseline.h5ad")b

In [111]:
adata = adata_train[adata_train.obs.dataset == "Wang"]

In [112]:
adata.X.sum()

83700230.0

In [109]:
sc.pp.log1p(adata)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:373: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [110]:
adata.X.sum()

56822120.0